In [11]:
import pandas as pd 
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
import numpy as np 
from optbinning import OptimalBinning
import warnings
warnings.filterwarnings('ignore')

In [12]:
df = pd.read_parquet(r'C:\Users\PC\Documents\GitHub\Khoa-luan\train.parquet')
oos = pd.read_parquet(r'C:\Users\PC\Documents\GitHub\Khoa-luan\oos.parquet')
oot = pd.read_parquet(r'C:\Users\PC\Documents\GitHub\Khoa-luan\oot.parquet')
df.columns

Index(['C_GIOITINH', 'TRINHDO', 'TTHONNHAN', 'SOHUUNHA', 'NHANVIENBIDV',
       'BASE_AUM', 'TUOI', 'INCOME', 'CBAL', 'CBALORG', 'AFLIMT_MAX',
       'AFLIMT_MIN', 'AFLIMT_AVG', 'CBAL_AVG', 'CBAL_MAX', 'CBAL_MIN',
       'COLLATERAL_VALUE', 'LTV', 'N_AVG_DEPOSIT_12M', 'N_AVG_DEPOSIT_6M',
       'N_AVG_DD_12M', 'N_AVG_CD_12M', 'FLAG_SALARY_ACC', 'FLAG_DEPOSIT',
       'UTILIZATION_RATE', 'CNT_CREDIT_CARDS', 'AMT_CASH_ADVANCE_12M',
       'FLAG_CASH_ADVANCE', 'PCT_PAYMENT_TO_BALANCE', 'CNT_MIN_PAY_6M',
       'AVG_DAYS_PAST_DUE', 'DTI_RATIO', 'PTI_RATIO', 'MOB',
       'CNT_OTHER_PRODUCTS', 'LIMIT_TO_INCOME', 'AMT_VAR_6M',
       'CBAL_SHORTTERM_LOAN', 'CBAL_LONGTERM_LOAN', 'CNT_DPD_30PLUS_6M',
       'OCCUPATION_TYPE', 'DURATION_MAX', 'REMAINING_DURATION_MAX',
       'TIME_TO_OP_MAX', 'RATE_AVG', 'PURCOD_MAX', 'PURCOD_MIN', 'MAX_DPD_12M',
       'MAX_DPD_12M_OBS', 'AVG_OD_DPD_12M', 'SUM_ALL_OD_12M',
       'N_AVG_OVERDUE_CBAL_12M', 'CBAL_TO_INC_12MON', 'REAL_GDP_GROWTH_12M',
       'BAD

In [13]:
segmentation_candidates = [
    # --- NHÓM 1: QUY MÔ & TÀI SẢN (Wealth & Scale) ---
    'BASE_AUM',             # Tổng tài sản
    'INCOME',               # Thu nhập
    'COLLATERAL_VALUE',     # Giá trị TSBĐ (Key Driver)
    'SOHUUNHA',             # Có nhà?
    
    # --- NHÓM 2: CẤU TRÚC TÍN DỤNG (Credit Structure) ---
    'AFLIMT_MAX',           # Hạn mức (Sức chứa)
    'CBAL',                 # Dư nợ hiện tại (Quy mô nợ)
    'LTV',                  # Đòn bẩy Tài sản (Key Driver)
    'DTI_RATIO',            # [MỚI] Đòn bẩy Thu nhập (Quan trọng cho nhóm Tín chấp)
    
    # --- NHÓM 3: ĐẶC ĐIỂM KHOẢN VAY (Product Features) ---
    'PURCOD_MAX',           # [MỚI] Mục đích vay (Mua nhà/Tiêu dùng?)
    'DURATION_MAX',         # [MỚI] Kỳ hạn vay (Ngắn/Dài)
    'MOB',                  # [MỚI] Thâm niên khách hàng (New/Old)
    
    # --- NHÓM 4: THANH KHOẢN (Liquidity) ---
    'N_AVG_DEPOSIT_12M',    # Tiền gửi bình quân (Key Driver)
    'FLAG_SALARY_ACC',      # Tài khoản lương
    'FLAG_DEPOSIT',         # Có gửi tiết kiệm ko?
    
    # --- NHÓM 5: NHÂN KHẨU (Demographics) ---
    'TUOI',                 
    'TRINHDO',              
    'TTHONNHAN',
    'OCCUPATION_TYPE'       # [MỚI] Nghề nghiệp
]

valid_candidates = [c for c in segmentation_candidates if c in df.columns]
X_global = df[valid_candidates].copy()
y_global = df['BAD_NEXT_12M'].copy()

X_global.info()
cat_cols = X_global.select_dtypes(include=['object', 'category']).columns
enc = OrdinalEncoder() # chỉ có TTHONNHAN là biến object và ordial encode là phù hợp 
X_global[cat_cols] = enc.fit_transform(X_global[cat_cols].fillna('Unknown')) # hàm encoder không chấp nhận input có null nên phải fill trước khi cho vào.Nhưng cột này không null gì cả 

cols_fill_zero = ['INCOME', 'AMT_VAR_6M', 'COLLATERAL_VALUE']
# Chỉ lấy các cột thực sự có trong data để tránh lỗi
cols_fill_zero = [c for c in cols_fill_zero if c in X_global.columns]
X_global[cols_fill_zero] = X_global[cols_fill_zero].fillna(0)

# Nhóm 2: Fill -1 (Trình độ)
if 'TRINHDO' in X_global.columns:
    X_global['TRINHDO'] = X_global['TRINHDO'].fillna(-1)

# Nhóm 3: Fill nốt các cột còn lại (để tránh lỗi crash Decision Tree)
# Vì sklearn không chấp nhận bất kỳ NaN nào
X_global = X_global.fillna(0) 

print("Số lượng Missing còn lại:", X_global.isnull().sum().sum())

dt_model = DecisionTreeClassifier(
    criterion='gini',
    splitter='best',
    class_weight='balanced',
    random_state=42,
    #max_depth=5  # Giới hạn độ sâu để tránh overfitting khi tính độ quan trọng
)

# Huấn luyện mô hình
dt_model.fit(X_global, y_global)

# 5. Trích xuất Feature Importance
importance_df = pd.DataFrame({
    'Tên Biến': segmentation_candidates,
    'Độ Quan Trọng (Feature Importance)': dt_model.feature_importances_
})

# Sắp xếp từ cao xuống thấp
importance_df = importance_df.sort_values(by='Độ Quan Trọng (Feature Importance)', ascending=False)

print("--- KẾT QUẢ XẾP HẠNG ĐỘ QUAN TRỌNG CỦA BIẾN ---")
pd.reset_option('display.float_format')
print(importance_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1137807 entries, 0 to 1137806
Data columns (total 18 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   BASE_AUM           1137807 non-null  int64  
 1   INCOME             1080709 non-null  float64
 2   COLLATERAL_VALUE   917388 non-null   float64
 3   SOHUUNHA           1137807 non-null  int64  
 4   AFLIMT_MAX         1137807 non-null  int64  
 5   CBAL               1137807 non-null  int64  
 6   LTV                1137807 non-null  float64
 7   DTI_RATIO          1137807 non-null  float64
 8   PURCOD_MAX         1137807 non-null  int64  
 9   DURATION_MAX       1137807 non-null  int64  
 10  MOB                1137807 non-null  int32  
 11  N_AVG_DEPOSIT_12M  1137807 non-null  int64  
 12  FLAG_SALARY_ACC    1137807 non-null  int64  
 13  FLAG_DEPOSIT       1137807 non-null  int64  
 14  TUOI               1137807 non-null  int64  
 15  TRINHDO            1103703 non-n

Các biến có độ quan trọng cao: . 2 biến được lựa chọn để phân khúc vừa mạnh, vừa đáp ứng được business logic: 
- N_AVG_DEPOSIT_12M: Tiền gửi bình quân 12 tháng
- DTI_RATIO: Tỷ lệ Nợ trên Thu nhập

In [14]:
# ==========================================
# 11. TÌM NGƯỠNG CẮT TỐI ƯU & TỰ ĐỘNG LẤY SỐ LÀM TRÒN
# ==========================================


print("\n=== TÌM NGƯỠNG CẮT TỐI ƯU (AUTO BUSINESS THRESHOLD) ===")

def get_auto_threshold(df, feature, target, round_digits=0):
    """
    Hàm tìm điểm cắt tối ưu và tự động làm tròn theo quy tắc kinh doanh.
    Trả về: Giá trị đã làm tròn (Business Threshold)
    """
    # 1. Chạy Optimal Binning
    optb = OptimalBinning(name=feature, prebinning_method='cart', divergence='iv', dtype="numerical", solver="mip", max_n_bins=2, min_n_bins=2, min_bin_size=0.1)
    optb.fit(df[feature], df[target])
    
    splits = optb.splits
    
    print(f"\n--- Phân tích biến: {feature} ---")
    
    if len(splits) > 0:
        raw_cutoff = splits[0]
        
        # 2. Logic làm tròn tự động
        business_cutoff = round(raw_cutoff, round_digits)
        
        # Định dạng in ấn cho đẹp
        if round_digits < 0: fmt = ",.0f" # Format tiền (1,000,000)
        else: fmt = f".{abs(round_digits)}f" # Format tỷ lệ (1.50)

        print(f"-> Điểm cắt Toán học (Raw): {raw_cutoff:,.4f}")
        print(f"-> Điểm cắt Kinh doanh (Business): {business_cutoff:{fmt}} (Đã lưu)")
        
        # In bảng chi tiết để double check
        binning_table = optb.binning_table.build()
        display(binning_table[['Bin', 'Count', 'Event rate', 'IV']])
        
        return business_cutoff # <--- TRẢ VỀ GIÁ TRỊ ĐÃ LÀM TRÒN
    else:
        print("-> Không tìm thấy điểm cắt tối ưu.")
        return None

# ==========================================
# THỰC THI (CHẠY MỘT LẦN RA LUÔN BIẾN)
# ==========================================

# 1. Tìm & Lưu ngưỡng TIỀN GỬI (Tự động làm tròn đến hàng Triệu: -6)
THRESH_DEPOSIT = get_auto_threshold(
    df, 'N_AVG_DEPOSIT_12M', 'BAD_NEXT_12M', round_digits=-6
)

# 2. Lọc dữ liệu tự động bằng ngưỡng vừa tìm được
# (Nếu không tìm được ngưỡng thì lấy mặc định là 0)
cutoff_val = THRESH_DEPOSIT if THRESH_DEPOSIT is not None else 0
df_mass = df[df['N_AVG_DEPOSIT_12M'] < cutoff_val].copy()

# 3. Tìm & Lưu ngưỡng DTI (Tự động làm tròn 1 số lẻ: 1)
THRESH_COLLATERAL = get_auto_threshold(
    df_mass, 'COLLATERAL_VALUE', 'BAD_NEXT_12M', round_digits=-8
)

# ==========================================
# KẾT QUẢ CUỐI CÙNG (DÙNG ĐỂ APPLY LUÔN)
# ==========================================
print("\n" + "="*30)
print("   CÁC THAM SỐ ĐÃ ĐƯỢC CHỐT TỰ ĐỘNG")
print("="*30)
print(f"THRESH_DEPOSIT = {THRESH_DEPOSIT:,.0f}")
print(f"THRESH_DTI     = {THRESH_COLLATERAL}")


=== TÌM NGƯỠNG CẮT TỐI ƯU (AUTO BUSINESS THRESHOLD) ===

--- Phân tích biến: N_AVG_DEPOSIT_12M ---
-> Điểm cắt Toán học (Raw): 1,663,807.5000
-> Điểm cắt Kinh doanh (Business): 2,000,000 (Đã lưu)


,Bin,Count,Event rate,IV
0,"(-inf, 1663807.50)",578099,0.124233,0.207322
1,"[1663807.50, inf)",559708,0.023187,0.447282
2,Special,0,0.000000,0.000000
3,Missing,0,0.000000,0.000000
Totals,,1137807,0.074527,0.654604



--- Phân tích biến: COLLATERAL_VALUE ---
-> Điểm cắt Toán học (Raw): 2,000,099,648.0000
-> Điểm cắt Kinh doanh (Business): 2,000,000,000 (Đã lưu)


,Bin,Count,Event rate,IV
0,"(-inf, 2000099648.00)",309801,0.150329,0.046311
1,"[2000099648.00, inf)",201611,0.040653,0.262108
2,Special,0,0.000000,0.000000
3,Missing,132525,0.153065,0.022968
Totals,,643937,0.116553,0.331386



   CÁC THAM SỐ ĐÃ ĐƯỢC CHỐT TỰ ĐỘNG
THRESH_DEPOSIT = 2,000,000
THRESH_DTI     = 2000000000.0


In [15]:
SEG_MAP = {
    1: '1. Prime (High Liquidity)',
    2: '2. Standard (Low DTI)',
    3: '3. Subprime (High Risk)'
}

# ---------------------------------------------------------
# 2. HÀM XỬ LÝ (CORE FUNCTIONS)
# ---------------------------------------------------------
def process_segmentation_and_save(df, name_prefix, save=True):
    """
    Hàm thực hiện trọn gói: Gán nhãn -> In thống kê -> Tách file -> Lưu file
    """
    data = df.copy()
    
    # --- A. GÁN NHÃN PHÂN KHÚC (SEGMENTATION LOGIC) ---
    conditions = [
        (data['N_AVG_DEPOSIT_12M'] >= THRESH_DEPOSIT), # Nhóm 1: Giàu
        (data['N_AVG_DEPOSIT_12M'] < THRESH_DEPOSIT) & (data['COLLATERAL_VALUE'] >= THRESH_COLLATERAL) # Nhóm 2: Khá
    ]
    choices = [SEG_MAP[1], SEG_MAP[2]]
    
    # Nhóm 3 là default (Bao gồm cả NaN, DTI cao...)
    data['SEGMENT'] = np.select(conditions, choices, default=SEG_MAP[3])
    
    # --- B. IN THỐNG KÊ (REPORT) ---
    print(f"\n>> Đang xử lý tập: {name_prefix.upper()} ({len(data):,} records)")
    print(data['SEGMENT'].value_counts(normalize=True).sort_index().apply(lambda x: f"{x:.1%}"))
    
    # --- C. TÁCH VÀ LƯU FILE (SPLIT & SAVE) ---
    if save:
        for seg_name in SEG_MAP.values():
            # Lọc dữ liệu
            sub_df = data[data['SEGMENT'] == seg_name]
            
            # Tạo tên file: train_seg1.parquet, oos_seg2.parquet...
            # Rút gọn tên file cho đẹp: '1. Prime...' -> 'seg1'
            seg_code = 'seg1' if '1.' in seg_name else 'seg2' if '2.' in seg_name else 'seg3'
            file_name = f"{name_prefix}_{seg_code}.parquet"
            
            # Lưu file
            sub_df.to_parquet(file_name, index=False)
            # print(f"   - Đã lưu: {file_name} ({len(sub_df):,} dòng)")
            
    return data # Trả về df đã gán nhãn để dùng tiếp nếu cần

# ---------------------------------------------------------
# 3. THỰC THI (EXECUTION)
# ---------------------------------------------------------

# Chạy lần lượt cho 3 tập dữ liệu
# Lưu ý: Biến df, oos, oot phải được load từ trước
df  = process_segmentation_and_save(df, 'train')
oos = process_segmentation_and_save(oos, 'oos')
oot = process_segmentation_and_save(oot, 'oot')

# ---------------------------------------------------------
# 4. KIỂM TRA CHẤT LƯỢNG KỸ LƯỠNG (VALIDATION ON TRAIN)
# ---------------------------------------------------------
print("\n" + "="*40)
print("   KIỂM TRA HIỆU QUẢ PHÂN KHÚC (TẬP TRAIN)")
print("="*40)

summary = df.groupby('SEGMENT').agg({
    'BAD_NEXT_12M': ['count', 'mean'],
    'N_AVG_DEPOSIT_12M': 'mean',
    'DTI_RATIO': 'mean',
    'COLLATERAL_VALUE': 'mean'
})

# Đổi tên cột hiển thị cho chuyên nghiệp
summary.columns = ['Count', 'Bad Rate', 'Deposit Mean', 'DTI Mean', 'Collateral Mean']
pd.options.display.float_format = '{:,.2f}'.format

# Hiển thị bảng
print(summary)

print("\n✅ HOÀN TẤT! Đã lưu đủ 9 files (Train/OOS/OOT x 3 Segments).")


>> Đang xử lý tập: TRAIN (1,137,807 records)
SEGMENT
1. Prime (High Liquidity)    43.4%
2. Standard (Low DTI)        17.7%
3. Subprime (High Risk)      38.9%
Name: proportion, dtype: object

>> Đang xử lý tập: OOS (300,317 records)
SEGMENT
1. Prime (High Liquidity)    47.9%
2. Standard (Low DTI)        16.0%
3. Subprime (High Risk)      36.1%
Name: proportion, dtype: object

>> Đang xử lý tập: OOT (302,113 records)
SEGMENT
1. Prime (High Liquidity)    49.7%
2. Standard (Low DTI)        15.5%
3. Subprime (High Risk)      34.8%
Name: proportion, dtype: object

   KIỂM TRA HIỆU QUẢ PHÂN KHÚC (TẬP TRAIN)
                            Count  Bad Rate  Deposit Mean  DTI Mean  \
SEGMENT                                                               
1. Prime (High Liquidity)  493870      0.02  7,382,858.36      1.27   
2. Standard (Low DTI)      201630      0.04    932,863.70      1.33   
3. Subprime (High Risk)    442307      0.15    860,998.51      1.32   

                           Collater